## Import Module

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [4]:
client = RemoteAPIClient()

sim = client.require('sim')

In [5]:
joints = []
# Get joint handles

for i in range(5):
    joints.append(sim.getObject(f"/youBotArmJoint{i}"))

# Gripper joint
joints.append(sim.getObject("/youBotGripperJoint2"))

for joint in joints:
    sim.setObjectInt32Param(
        joint,
        sim.jointintparam_dynctrlmode,
        sim.jointdynctrl_position,
    )

In [6]:
points = []
# Car 위치
points.append(sim.getObject(f"/youBot_ref"))
# 5개의 관절 위치 가져오기
for i in range(5):
    points.append(sim.getObject(f"/p{i}_ref"))
# End Effector 위치
points.append(sim.getObject(f"/pe_ref"))



In [39]:
# Define the DH parameters
DH_params = [
    [0.0330, np.pi/2, 0.1012, 0],  # Joint 1: [r1, alpha1, d1, theta1 (q1)]
    [0.1550, 0, 0.0000, 0],        # Joint 2: [r2, alpha2, d2, theta2 (q2)]
    [0.1348, 0, 0.0000, 0],        # Joint 3: [r3, alpha3, d3, theta3 (q3)]
    [0.0005, -np.pi/2, 0.0190, 0], # Joint 4: [r4, alpha4, d4, theta4 (q4)]
    [0.0006, 0, 0.0000, 0],        # Joint 5: [r5, alpha5, d5, theta5 (q5)]
]


In [40]:
def dh_transformation(a, alpha, d, theta):
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

In [41]:
def forward_kinematics(DH_params, joint_angles):
    T = np.eye(4)  # 초기 변환 행렬 (항등 행렬)
    for i in range(len(DH_params)):
        a, alpha, d, _ = DH_params[i]
        theta = joint_angles[i]
        T_i = dh_transformation(a, alpha, d, theta)  # 각 관절의 변환 행렬 계산
        T = T @ T_i  # 전체 변환 행렬에 곱하기
    return T

In [42]:
def run_step():
    # 임의의 각도 값 (radian 단위로 설정)
    target_angles = [np.pi, np.pi/4, np.pi/3, np.pi/6, np.pi/4]  # 원하는 각도 값

    # 각 관절에 임의의 각도 값을 설정
    for i, joint in enumerate(joints[:5]):  # 그리퍼를 제외한 5개의 관절
        sim.setJointTargetPosition(joint, target_angles[i])

    # Forward Kinematics 계산을 위한 각도 값 가져오기
    joint_angles = []
    for joint in joints[:5]:
        angle = sim.getJointPosition(joint)
        joint_angles.append(angle)

    # Forward Kinematics 계산
    T_end_effector = forward_kinematics(DH_params, joint_angles)

    # End Effector의 위치 추출
    end_effector_position = T_end_effector[:3, 3]

    # 실제 End Effector의 위치 가져오기
    actual_position = sim.getObjectPosition(points[6])  # points[6]는 End Effector의 실제 위치 (pe_ref)

    # 실제 위치와 FK 계산 위치 간의 거리 계산
    distance = np.linalg.norm(np.array(end_effector_position) - np.array(actual_position))

    print(f"End-effector position by FK: {end_effector_position}")
    print(f"End-effector position by simulation: {actual_position}")

    # 목표 위치에 도달했는지 확인 (거리 차이가 작은 경우 목표 도달)
    tolerance = 1  # 허용 오차 (1cm)
    if distance < tolerance:
        print("goal-reached!")
        return True  # 목표에 도달하면 True 반환
    
    return False  # 목표에 아직 도달하지 않으면 False 반환


In [48]:
n_steps = 1000  # 최대 스텝 수를 설정해줍니다.

# Run simulation
sim.setStepping(True)
sim.startSimulation()

for i in range(n_steps):
    goal_reached = run_step()
    if goal_reached:  # 목표에 도달했으면 시뮬레이션 종료
        break


sim.stopSimulation()


End-effector position by FK: [ 0.3239 -0.019   0.1012]
End-effector position by simulation: [-9.999999999996627e-05, 1.1991800000000001, 0.7399999999999997]
End-effector position by FK: [ 0.3239 -0.019   0.1012]
End-effector position by simulation: [-9.999999999996627e-05, 1.1991800000000001, 0.7399999999999997]
End-effector position by FK: [ 0.3239 -0.019   0.1012]
End-effector position by simulation: [-9.999999999996627e-05, 1.1991800000000001, 0.7399999999999997]
End-effector position by FK: [ 0.3239 -0.019   0.1012]
End-effector position by simulation: [-9.999999999996627e-05, 1.1991800000000001, 0.7399999999999997]
End-effector position by FK: [ 0.3239 -0.019   0.1012]
End-effector position by simulation: [-9.999999999996627e-05, 1.1991800000000001, 0.7399999999999997]
End-effector position by FK: [ 0.3239 -0.019   0.1012]
End-effector position by simulation: [-9.999999999996627e-05, 1.1991800000000001, 0.7399999999999997]
End-effector position by FK: [ 0.3239 -0.019   0.1012]
End